In [1]:
import selenium
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
import warnings
warnings.filterwarnings('ignore')
import time
import requests
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

### 1. Scrape the details of most viewed videos on YouTube from Wikipedia.
Url = https://en.wikipedia.org/wiki/List_of_most-viewed_YouTube_videos You need to find following details:
* A) Rank
* B) Name
* C) Artist
* D) Upload date
* E) Views

In [19]:
driver = webdriver.Chrome(r'chromedriver.exe')
driver.get('https://en.wikipedia.org/wiki/List_of_most-viewed_YouTube_videos')
soup = BeautifulSoup(driver.page_source, 'lxml')
tables = soup.find_all('table')
most_viewed = pd.read_html(driver.page_source)[1].set_index('No.').iloc[:,:4]
most_viewed.head(5)

,Video name,Uploader,Views (billions),Publication date
No.,,,,
1.,"""Baby Shark Dance""[4]",Pinkfong Baby Shark - Kids' Songs & Stories,11.98,"June 17, 2016"
2.,"""Despacito""[7]",Luis Fonsi,8.04,"January 12, 2017"
3.,"""Johny Johny Yes Papa""[14]",LooLoo Kids,6.56,"October 8, 2016"
4.,"""Bath Song""[15]",Cocomelon – Nursery Rhymes,5.88,"May 2, 2018"
5.,"""Shape of You""[16]",Ed Sheeran,5.87,"January 30, 2017"


### 
2. Scrape the details team India’s international fixtures from bcci.tv. Url = https://www.bcci.tv/.
You need to find following details:
* A) Match title (I.e. 1st ODI)
* B) Series
* C) Place
* D) Date
* E) Time
* Note: - From bcci.tv home page you have reach to the international fixture page through code.

In [58]:
driver = webdriver.Chrome(r'chromedriver.exe')
driver.get('https://www.bcci.tv/international/fixtures')

In [59]:
while True:
    try:
        loadMoreButton = driver.find_element_by_xpath("/html/body/div[2]/div[2]/div/div/div/div[2]/div[3]/div[2]/div/button")
        time.sleep(2)
        loadMoreButton.click()
        time.sleep(5)
    except Exception as e:
        print (e)
        break
print ("Complete")
time.sleep(10)

Message: element click intercepted: Element <button class="match-btn btn-red d-flex align-items-center justify-content-center mx-auto mt-3" ng-click="viewMoreMatches()">...</button> is not clickable at point (508, 695). Other element would receive the click: <div class="cookie no-disply" style="display: block;">...</div>
  (Session info: chrome=108.0.5359.125)
Stacktrace:
Backtrace:
	Ordinal0 [0x0084ACD3+2075859]
	Ordinal0 [0x007DEE61+1633889]
	Ordinal0 [0x006DB7BD+571325]
	Ordinal0 [0x00711499+791705]
	Ordinal0 [0x0070F4AC+783532]
	Ordinal0 [0x0070D0AB+774315]
	Ordinal0 [0x0070BD37+769335]
	Ordinal0 [0x00701C76+728182]
	Ordinal0 [0x0072731C+881436]
	Ordinal0 [0x007015BF+726463]
	Ordinal0 [0x00727534+881972]
	Ordinal0 [0x0073B56A+963946]
	Ordinal0 [0x00727136+880950]
	Ordinal0 [0x006FFEFD+720637]
	Ordinal0 [0x00700F3F+724799]
	GetHandleVerifier [0x00AFEED2+2769538]
	GetHandleVerifier [0x00AF0D95+2711877]
	GetHandleVerifier [0x008DA03A+521194]
	GetHandleVerifier [0x008D8DA0+516432]
	Ord

In [55]:
desc = driver.find_elements(By.XPATH, '//div[contains(@class,"fixture-card ng-scope border-orange") or contains(@class,"fixture-card ng-scope border-sky") or contains(@class,"fixture-card ng-scope border-green")]')
data = pd.DataFrame([i.text for i in desc], columns = ['desc'])
data = data['desc'].str.split('\n', expand=True).rename(columns = {0:'Series', 1:'Date', 2:'Time', 3:'Team_1', 5:'Team_2', 6:'place'})
data.drop([4,7], axis = 1, inplace = True)


In [62]:
place= data['place'].str.split('-', expand=True).rename(columns = {0:'Match_Title', 1:'Place'}).drop([2], axis = 1)
BCCI_data = pd.concat([data, place], axis = 1).drop(['place'], axis = 1)

,Series,Date,Time,Team_1,Team_2,Match_Title,Place
0,INDIA WOMEN U19 TOUR OF SOUTH AFRICA WOMEN U19,29 DEC 2022,5:15 PM IST,India Women U19,South Africa Women U19,2nd T20I,"Steyn City School Ground, Pretoria"
1,INDIA WOMEN U19 TOUR OF SOUTH AFRICA WOMEN U19,31 DEC 2022,1:30 PM IST,India Women U19,South Africa Women U19,3rd T20I,"Steyn City School Ground, Pretoria"
2,INDIA WOMEN U19 TOUR OF SOUTH AFRICA WOMEN U19,2 JAN 2023,5:15 PM IST,India Women U19,South Africa Women U19,4th T20I,"Steyn City School Ground, Pretoria"
3,SRI LANKA TOUR OF INDIA T20 SERIES 2022-23,3 JAN 2023,7:00 PM IST,India,Sri Lanka,1st T20I,"Wankhede Stadium, Mumbai"
4,INDIA WOMEN U19 TOUR OF SOUTH AFRICA WOMEN U19,4 JAN 2023,1:30 PM IST,India Women U19,South Africa Women U19,5th T20I,"Steyn City School Ground, Pretoria"
5,SRI LANKA TOUR OF INDIA T20 SERIES 2022-23,5 JAN 2023,7:00 PM IST,India,Sri Lanka,2nd T20I,"Maharashtra Cricket Association Stadium, Pune"
6,SRI LANKA TOUR OF INDIA T20 SERIES 2022-23,7 JAN 2023,7:00 PM IST,India,Sri Lanka,3rd T20I,"Saurashtra Cricket Association Stadium, Rajkot"
7,SRI LANKA TOUR OF INDIA ODI SERIES 2022-23,10 JAN 2023,1:30 PM IST,India,Sri Lanka,1st ODI,"Barsapara Cricket Stadium, Guwahati"
8,SRI LANKA TOUR OF INDIA ODI SERIES 2022-23,12 JAN 2023,1:30 PM IST,India,Sri Lanka,2nd ODI,"Eden Gardens, Kolkata"
9,SRI LANKA TOUR OF INDIA ODI SERIES 2022-23,15 JAN 2023,1:30 PM IST,India,Sri Lanka,3rd ODI,"Greenfield International Stadium, Thiruvanant..."


### 
Scrape the details of selenium exception from guru99.com. Url = https://www.guru99.com/
You need to find following details:
* A) Name
* B) Description
* Note: - From guru99 home page you have to reach to selenium exception handling page through code.

In [76]:
driver = webdriver.Chrome(r'chromedriver.exe')
driver.get('https://www.guru99.com/')
time.sleep(10)
search = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div/main/div/article/div/div[1]/div[1]/div[2]/div/div[1]/div/div/div/form/table/tbody/tr/td[1]/div/table/tbody/tr/td[1]/input')
search.send_keys('selenium exception handling')
butn = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div/main/div/article/div/div[1]/div[1]/div[2]/div/div[1]/div/div/div/form/table/tbody/tr/td[2]/button')
butn.click()
time.sleep(5)
click = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div/main/div/article/div/div[1]/div[1]/div[2]/div/div[1]/div/div/div/div[1]/div[6]/div[2]/div/div/div[1]/div[1]/div[1]/div[1]/div/a')
click.click()

### 
* Scrape the details of State-wise GDP of India from statisticstime.com. Url = http://statisticstimes.com/
You have to find following details:
* A) Rank
* B) State
* C) GSDP(18-19)- at current prices
* D) GSDP(19-20)- at current prices
* E) Share(18-19)
* F) GDP($ billion)

In [73]:
driver = webdriver.Chrome(r'chromedriver.exe')
driver.get('https://www.statisticstimes.com/economy/india/indian-states-gdp.php')
soup = BeautifulSoup(driver.page_source, 'lxml')
#tables = soup.find_all('table')
df = pd.read_html(driver.page_source)[1]

In [76]:
gdp_india = df.iloc[:, 0:6]


In [79]:
gdp_india.head(5)

Rank          State GSDP (Cr INR at Current prices)            Share  \
  Rank          State                           19-20    18-19   18-19   
0  1.0    Maharashtra                               -  2632792  13.94%   
1  2.0     Tamil Nadu                         1845853  1630208   8.63%   
2  3.0  Uttar Pradesh                         1687818  1584764   8.39%   
3  4.0        Gujarat                               -  1502899   7.96%   
4  5.0      Karnataka                         1631977  1493127   7.91%   

  GDP ($billion)  
            2019  
0        399.921  
1        247.629  
2        240.726  
3        228.290  
4        226.806

In [72]:
df.loc[:2, (slice(None),'19-20')]

,GSDP (Cr INR at Current prices),GSDP (Cr INR at 2011-12 prices)
,19-20,19-20
0,-,-
1,1845853,1312929
2,1687818,1166817


### 
* Scrape the details of trending repositories on Github.com. Url = https://github.com/
You have to find the following details:
* A) Repository title
* B) Repository description
* C) Contributors count
* D) Language used

* Note: - From the home page you have to click on the trending option from Explore menu through code.

In [155]:

driver = webdriver.Chrome(r'chromedriver.exe')
driver.get('https://github.com/')
time.sleep(5)
butn = driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/header/div/div[2]/div/nav/ul/li[3]/button')
butn.click()
butn = driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/header/div/div[2]/div/nav/ul/li[3]/div/ul[3]/li[3]/a')
butn.click()

In [144]:
lists = [] 

for page in range(1,2):
    
    try:
        name = driver.find_elements(By.XPATH, '//h1[@class="h3 lh-condensed"]//a')
        lang = driver.find_elements(By.XPATH, '//span[@itemprop="programmingLanguage"]')
        desc = driver.find_elements(By.XPATH, '//a[@class="Link--muted d-inline-block mr-3"]')
        discription = driver.find_elements(By.XPATH, '//p[@class="col-9 color-fg-muted my-1 pr-4"]')
          
        
        
    except NoSuchElementException:
                Name.append('_')
                Lang.append('_')
                Desc.append('_')
                Discription.append('_')

### 
* Scrape the details of top 100 songs on billiboard.com. Url = https:/www.billboard.com/
You have to find the following details:
* A) Song name
* B) Artist name
* C) Last week rank
* D) Peak rank
* E) Weeks on board
* Note: - From the home page you have to click on the charts option then hot 100-page link through code.

In [174]:
driver = webdriver.Chrome(r'chromedriver.exe')
driver.get('https://www.billboard.com/')
#time.sleep(5)
butn = driver.find_element(By.XPATH, '/html/body/div[3]/header/div/div[2]/div/div/div[2]/div[2]/div/div/nav/ul/li[1]/a')
butn.click()
time.sleep(5)
button = driver.find_element(By.XPATH, '/html/body/div[3]/main/div[2]/div[3]/div/div/div[2]/div/div[1]/a')
button.click()


ElementClickInterceptedException: Message: element click intercepted: Element is not clickable at point (137, 3171)
  (Session info: chrome=108.0.5359.125)
Stacktrace:
Backtrace:
	Ordinal0 [0x0104ACD3+2075859]
	Ordinal0 [0x00FDEE61+1633889]
	Ordinal0 [0x00EDB7BD+571325]
	Ordinal0 [0x00F11499+791705]
	Ordinal0 [0x00F0F4AC+783532]
	Ordinal0 [0x00F0D0AB+774315]
	Ordinal0 [0x00F0BD37+769335]
	Ordinal0 [0x00F01C76+728182]
	Ordinal0 [0x00F2731C+881436]
	Ordinal0 [0x00F015BF+726463]
	Ordinal0 [0x00F27534+881972]
	Ordinal0 [0x00F3B56A+963946]
	Ordinal0 [0x00F27136+880950]
	Ordinal0 [0x00EFFEFD+720637]
	Ordinal0 [0x00F00F3F+724799]
	GetHandleVerifier [0x012FEED2+2769538]
	GetHandleVerifier [0x012F0D95+2711877]
	GetHandleVerifier [0x010DA03A+521194]
	GetHandleVerifier [0x010D8DA0+516432]
	Ordinal0 [0x00FE682C+1665068]
	Ordinal0 [0x00FEB128+1683752]
	Ordinal0 [0x00FEB215+1683989]
	Ordinal0 [0x00FF6484+1729668]
	BaseThreadInitThunk [0x769A00F9+25]
	RtlGetAppContainerNamedObjectPath [0x77A17BBE+286

In [159]:
soup = BeautifulSoup(driver.page_source, 'lxml')
#tables = soup.find_all('table')


In [171]:
desc = driver.find_elements('//ul[@class="lrv-a-unstyle-list lrv-u-flex lrv-u-height-100p lrv-u-flex-direction-column@mobile-max"]')

KeyboardInterrupt: 